In [1]:
import json
from tqdm import tqdm_notebook
from collections import Counter
import requests
from bs4 import BeautifulSoup
import pickle
import pandas as pd
from stepik_api import get_token

In [2]:
def get_data(pageNum):
    api_url = 'https://stepik.org/api/courses?page={}'.format(pageNum)
    course = json.loads(requests.get(api_url, headers={'Authorization': 'Bearer ' + get_token()}).text)
    return course

In [3]:
all_course = []

In [4]:
pbar = tqdm_notebook(range(1, 121))

for i in pbar:
    all_course += get_data(i)['courses']
    pbar.set_description(str(len(all_course)))
    

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


In [5]:
Counter([i['language'] for i in all_course])

Counter({'af': 1,
         'de': 2,
         'en': 291,
         'es': 12,
         'ko': 3,
         'pt-br': 1,
         'ru': 908,
         'uk': 8})

In [6]:
#pickle.dump(all_course, open("stepik_courses.pck", "wb"))

In [7]:
all_course = pickle.load(open("stepik_courses.pck", "rb"))

In [8]:
indexes = list(map(int, [i['id'] for i in all_course]))

$$$$$$$$

In [9]:
def parse_stepic(_id):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    html = requests.get("https://stepik.org/api/courses/%s/" % _id, headers=headers).text
    html = json.loads(html)
    
    print(html['courses'])

    text = html['courses'][0]['course_format'] + " "
    text += html['courses'][0]['description'] + " "
    text += html['courses'][0]['certificate'] + " "
    text += html['courses'][0]['requirements'] + " "
    text += html['courses'][0]['summary'] + " "
    text += html['courses'][0]['target_audience'] + " "
    text += html['courses'][0]['title'] + " "

    return [text, html['courses'][0]['title'], html['courses'][0]['is_active']]

In [10]:
parse_stepic(indexes[0])

[{'id': 3406, 'summary': 'Курс для подготовки к ЕГЭ. Типовые задания ', 'workload': '', 'cover': None, 'intro': '', 'course_format': '', 'target_audience': '', 'certificate_footer': None, 'certificate_cover_org': None, 'is_certificate_auto_issued': False, 'certificate_regular_threshold': None, 'certificate_distinction_threshold': None, 'instructors': [35159128], 'certificate': '', 'requirements': '', 'description': '', 'sections': [6968], 'total_units': 0, 'enrollment': None, 'is_favorite': False, 'actions': {}, 'progress': None, 'certificate_link': None, 'certificate_regular_link': None, 'certificate_distinction_link': None, 'schedule_link': None, 'schedule_long_link': None, 'first_deadline': None, 'last_deadline': None, 'subscriptions': ['31-78-3406', '30-78-3406'], 'announcements': [], 'is_contest': False, 'is_self_paced': True, 'is_adaptive': False, 'is_idea_compatible': False, 'last_step': '78-3406', 'intro_video': None, 'social_providers': [], 'authors': [35159128], 'tags': [], '

['    Курс для подготовки к ЕГЭ. Типовые задания   История России ЕГЭ  ',
 'История России ЕГЭ ']

In [14]:
texts = []
pbar = tqdm_notebook(indexes)

for i in pbar:
    try:
        text = parse_stepic(i)

        if text != None:
            texts.append(text + [i])
            pbar.set_description(str(len(texts)))
    except:
        pass

In [15]:
pickle.dump(texts, open('stepik.pck', 'wb'))

$$$$

In [36]:
def download_pack(start):
    url = 'https://www.coursera.org/courses?_facet_changed_=true&primaryLanguages=ru&start=%s' % start
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    html = requests.get(url, headers=headers).text
    root = BeautifulSoup(html, "html.parser")
    items = root.findAll('div', {'class': 'rc-SearchResults'})[0]
    urls = [x['href'] for x in items.findAll('a')]
    return urls

In [41]:
a = []

while len(a) < 116:
    a.extend(download_pack(len(a)))
    print('\r', len(a), end='')

 116

In [49]:
def text_sum(texts: list, separator: str = ' ') -> str:
    tmp = ''
    
    for text in texts:
        tmp += text + separator
    
    return tmp

In [105]:
def download_link(link):
    url = 'https://www.coursera.org%s' % link
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    html = requests.get(url, headers=headers)
    root = BeautifulSoup(html.content.decode(), "html.parser")
    
    title = root.findAll('h1', {'class': 'title'})
    
    if len(title) == 0:
        title = root.findAll('h2')[0]
    else:
        title = title[0]
    
    descriptions = root.findAll('p', {'class': 'course-description'})
    
    if len(descriptions) == 0:
        descriptions = root.findAll('div', {'class': 'description'})
    
    courses = root.findAll('div', {'class': 'course-cont'})
    
    if len(courses) == 0:
        courses = root.findAll('div', {'class': 'module-desc'})

    return [title.text, text_sum([description.text for description in descriptions]), 
            text_sum([course.text for course in courses]), url]

In [ ]:
answer = []

In [120]:
for link in tqdm_notebook(a):
    answer.append(download_link(link))

In [125]:
coursera_text = pd.DataFrame(answer, columns=['header', 'description', 'course_text', 'link'])

In [126]:
coursera_text.to_csv('coursera.csv')

# Благинин Data
https://openedu.ru/course/ <br>
http://universarium.org/catalog <br>
https://netology.ru/

In [5]:
url = "http://universarium.org/catalog"
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
html = requests.get(url, headers=headers)
root = BeautifulSoup(html.content.decode(), "html.parser")

In [7]:
root.findAll('a', {'ng-bind': 'course.title'})

[]

In [18]:
courses = open('data/courses.json').read()

In [25]:
courses = eval(courses[:-1].replace('false', 'False').replace('true','True'))

In [27]:
titles = []
urls = []

for i in courses:
    titles.append(courses[i]['title'])
    urls.append(courses[i]['url'])

In [28]:
data = []

for title, url in zip(titles, urls):
    data.append([title, url])

In [29]:
data = pd.DataFrame(data, columns=['title', 'url'])

In [45]:
def text_sum(text: list) -> str:
    tmp_text = ""
    for i in text:
        tmp_text += i
        tmp_text += " "
    return tmp_text

In [46]:
def parse_openedu(url):
    url = 'https://openedu.ru%s' % url
    html = requests.get(url, headers=headers)
    root = BeautifulSoup(html.content.decode(), "html.parser")
    needed_div = root.findAll('div', {'class': 'col-sm-8 issue'})[0]
    return text_sum([i.text for i in needed_div.findAll('p')])

In [48]:
data['text'] = list(map(parse_openedu, data['url']))

In [50]:
data['url'] = list(map(lambda url: 'https://openedu.ru%s' % url, data['url']))

In [51]:
data.head()

,title,url,text
0,Линейные электрические цепи,https://openedu.ru/course/ITMOUniversity/LINCIR/,В предлагаемом курсе изучаются статические и п...
1,Веб-программирование,https://openedu.ru/course/ITMOUniversity/WEBDEV/,Курс предназначен для новичков в области веб-п...
2,Управление мехатронными и робототехническими с...,https://openedu.ru/course/ITMOUniversity/ROBCTR/,"Длительное время люди изобретают механизмы, ко..."
3,Методы и алгоритмы теории графов,https://openedu.ru/course/ITMOUniversity/AGRAPH/,Теория графов – наиболее востребованный на пра...
4,Организационное поведение,https://openedu.ru/course/hse/ORGBH/,Курс представляет основные подходы и результат...


In [52]:
data.to_csv('openedu.csv')

In [53]:
url = "https://netology.ru/design/programs"
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
html = requests.get(url, headers=headers)
root = BeautifulSoup(html.content.decode(), "html.parser")